In [15]:
%pip install opacus

In [16]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
import time
import numpy as np

from opacus import PrivacyEngine
from opacus.utils.batch_memory_manager import BatchMemoryManager


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [18]:
!/opt/bin/nvidia-smi

Thu Apr 21 01:34:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    73W / 149W |   6488MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 超参数设置

In [19]:
EPOCH = 30 # 遍历数据集次数
BATCH_SIZE = 512  # 批处理尺寸(batch_size)
LR = 0.01  # 学习率
MAX_GRAD_NORM = 1.2
EPSILON = 2
DELTA = 1e-4
MAX_PHYSICAL_BATCH_SIZE = 256

# 获取数据

In [20]:
transform = transforms.ToTensor()
trainset = torchvision.datasets.MNIST(root='./dataset',train=True,download=True,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,shuffle=True,num_workers=0)

testset = torchvision.datasets.MNIST(root='./dataset',train=False,download=True,transform=transform)
testloader = torch.utils.data.DataLoader(testset,batch_size=BATCH_SIZE,shuffle=False,num_workers=0)


# 定义网络

In [21]:
class AlexNet(nn.Module):
    def __init__(self, width_mult=1):
        super(AlexNet, self).__init__()
        self.layer1 = nn.Sequential( # 输入1*28*28
            nn.Conv2d(1, 32, kernel_size=3, padding=1), # 32*28*28
            nn.MaxPool2d(kernel_size=2, stride=2), # 32*14*14
            nn.ReLU(inplace=True),
            )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1), # 64*14*14
            nn.MaxPool2d(kernel_size=2, stride=2), # 64*7*7
            nn.ReLU(inplace=True),
            )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1), # 128*7*7
            )
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1), # 256*7*7
            )
 
        self.layer5 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size=3, padding=1), # 256*7*7
            nn.MaxPool2d(kernel_size=3, stride=2), # 256*3*3
            nn.ReLU(inplace=True),
            )
        self.fc1 = nn.Linear(256*3*3, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 10)
 
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = x.view(-1, 256*3*3)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

# 定义网络损失函数优化器

该部分尝试自己改写梯度下降，但仿佛会让梯度爆炸，暂时先不用

In [22]:
# epsilon = 2

# # This analysis has a total privacy cost of epsilon = 1, even though we release many results!
# f = lambda x: x + np.random.laplace(loc=0, scale=1/epsilon)

# def mysgd(params, lr, batch_size):  
#     """小批量随机梯度下降"""
#     # print(params)
#     with torch.no_grad():
#         for param in params:
#             # print(param)
#             # param -= (lr * param.grad / batch_size).apply(f)
#             # param -= (lr * (param.grad+ torch.tensor(np.random.laplace(loc=0, scale=1/epsilon))) ) 
#             param -= (lr * param.grad) + torch.tensor(np.random.laplace(loc=0, scale=5/epsilon))

#             # param -= lr * param.grad / batch_size
#             param.grad.zero_()
#             # print(param)


In [23]:
net = AlexNet()
from opacus.validators import ModuleValidator

errors = ModuleValidator.validate(net, strict=False)
errors[-5:]

[]

In [24]:
net = ModuleValidator.fix(net)
ModuleValidator.validate(net, strict=False)
net = net.to(device)


In [25]:
criterion = nn.CrossEntropyLoss()  # 交叉熵损失函数，通常用于多分类问题上
optimizer = optim.SGD(net.parameters(), lr=LR, momentum=0.9)

In [26]:
privacy_engine = PrivacyEngine()

net, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=net,
    optimizer=optimizer,
    data_loader=trainloader,
    epochs=EPOCH,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)

print(f"Using sigma={optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")

/usr/local/lib/python3.7/dist-packages/opacus/privacy_engine.py:115: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "
/usr/local/lib/python3.7/dist-packages/opacus/accountants/analysis/rdp.py:333: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  f"Optimal order is the {extreme} alpha. Please consider expanding the range of alphas to get a tighter privacy bound."


Using sigma=1.1962890625 and C=1.2


# 训练

In [27]:
def train():
 
    for epoch in range(EPOCH):
        sum_loss = []
        net.train()
        with BatchMemoryManager(
            data_loader=trainloader, 
            max_physical_batch_size=MAX_PHYSICAL_BATCH_SIZE, 
            optimizer=optimizer
        ) as memory_safe_data_loader:
            # 数据读取
            for i, data in enumerate(memory_safe_data_loader):
                # 梯度清零
                optimizer.zero_grad() 
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
    
   
                # forward + backward
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                # mysgd(net.parameters(), lr=LR, batch_size=BATCH_SIZE)
    
                # 每训练100个batch打印一次平均loss
                sum_loss.append(loss.item())
                # if i % 100 == 99:
                #     print('[%d, %d] loss: %.03f'
                #         % (epoch + 1, i + 1, sum_loss / 100))
                #     sum_loss = 0.0
            # 每跑完一次epoch测试一下准确率
            net.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                for data in testloader:
                    images, labels = data
                    images, labels = images.to(device), labels.to(device)
                    outputs = net(images)
                    # 取得分最高的那个类
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum()
                epsilon = privacy_engine.get_epsilon(DELTA)
                print('第%d个epoch的识别准确率为：%d%%' % (epoch + 1, (100 * correct / total)),"loss为%f" % (np.mean(sum_loss)),f"(ε = {epsilon:.2f}, δ = {DELTA})")
            # 保存模型参数
            # torch.save(net.state_dict(), './params.pth')

In [28]:
train()

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


第1个epoch的识别准确率为：28% loss为2.289759 (ε = 0.55, δ = 0.0001)
第2个epoch的识别准确率为：59% loss为2.087370 (ε = 0.62, δ = 0.0001)
第3个epoch的识别准确率为：64% loss为1.369642 (ε = 0.69, δ = 0.0001)
第4个epoch的识别准确率为：69% loss为0.988036 (ε = 0.76, δ = 0.0001)
第5个epoch的识别准确率为：73% loss为0.875751 (ε = 0.82, δ = 0.0001)
第6个epoch的识别准确率为：76% loss为0.849529 (ε = 0.88, δ = 0.0001)
第7个epoch的识别准确率为：77% loss为0.826020 (ε = 0.94, δ = 0.0001)
第8个epoch的识别准确率为：79% loss为0.814218 (ε = 1.00, δ = 0.0001)
第9个epoch的识别准确率为：81% loss为0.810509 (ε = 1.06, δ = 0.0001)
第10个epoch的识别准确率为：81% loss为0.801823 (ε = 1.12, δ = 0.0001)
第11个epoch的识别准确率为：83% loss为0.791835 (ε = 1.17, δ = 0.0001)
第12个epoch的识别准确率为：84% loss为0.769097 (ε = 1.22, δ = 0.0001)
第13个epoch的识别准确率为：84% loss为0.758918 (ε = 1.27, δ = 0.0001)
第14个epoch的识别准确率为：85% loss为0.745147 (ε = 1.33, δ = 0.0001)
第15个epoch的识别准确率为：86% loss为0.728392 (ε = 1.37, δ = 0.0001)
第16个epoch的识别准确率为：87% loss为0.711224 (ε = 1.42, δ = 0.0001)
第17个epoch的识别准确率为：87% loss为0.691111 (ε = 1.47, δ = 0.0001)
第18个epoch的识别准确率为：88% lo